# Tutorial 2: Expressions containing NDArray objects

Python-Blosc2 implements a powerful way to operate with NDArray arrays and other objects, called "lazy expressions".  A lazy expression is a lightweight object which stores a desired computation symbolically, with references to its operands (stored on disk or in memory), but does not execute until data is explicitly requested, e.g. if a slice of the computation result is requested. The lazy expression will then execute, but only on the necessary portion of the data, making it especially efficient, and avoiding large in-memory computations.

In this tutorial, we will see how to do such lazy computations, which are especially useful when working with large arrays, owing to this avoidance of costly in-memory temporaries.


In [ ]:
!pip install ipympl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import blosc2
import time
import psutil
import os
%matplotlib ipympl

# --- Memory profiler ---
def getmem():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

## Part 1) A simple example
First, let's create a couple of NDArray arrays. We're going to write them to disk since in principle we are interested in large arrays (so big that they can't fit in memory).

In [ ]:
shape1 = (500, 10000)
shape2 = (10, 500, 10000)
a = blosc2.linspace(0, 1, np.prod(shape1), dtype=np.float32, shape=shape1, urlpath="a.b2nd", mode="w")
b = blosc2.linspace(1, 2, np.prod(shape2), dtype=np.float64, shape=shape2, urlpath="b.b2nd", mode="w")

# Now, let's create an expression that involves `a` and `b`, called `c`.
c = a**2 + b**2 + 2 * a * b + 1
print(c.info)  # at this stage, the expression has not been computed yet

We see that the type of `c` is a `LazyExpr` object.  This object is a placeholder for the actual computation that will be done when we compute the expression.  This is a very powerful feature because it allows us to build complex expressions without actually computing anything until we really need the result (or a portion of the result). 

**EXERCISE**: Moreover, we can access the individual metadata of the result (such as ``shape`` and `dtype`) ass sttributes of the lazy expression ``c`` rapidly, without computing anything. Time how long it takes (using the cell magic ``%time`` or line magic ``%%timeit``).

In [3]:
# YOUR CODE HERE

Before moving on, note that the shapes of `a` and `b` are different. However, the shapes are compatible, and so Blosc2 applies **broadcasting** (à la NumPy) efficiently, both to obtain the precomputation metadata (very fast) as well as to perform the computation (slower). 

Now, let's see all this in action in computation. `LazyExpr` objects follow the [LazyArray interface](../../reference/lazyarray.html), and this provides several ways for performing the computation, depending on the object we want as the desired output.

#### 1. Returning a NDArray array
**i) Computing an expression** First, let's use the `compute` method. The result will be another NDArray array:

In [ ]:
m0 = getmem()
t0 = time.time()
d = c.compute()
print(f"Computation time {round((time.time() - t0)*1000,1)} ms")
print(f"Memory used {round(getmem() - m0)} MB")
print(f"Class: {type(d)}, occupies {round(d.schunk.cbytes / 1024 ** 2)} MB. Compression ratio {d.schunk.cratio:.2f}x")
del d

Note that just accessing metadata via e.g. ``c.shape`` took much less time, since it uses the metadata of the operands (plus broadcasting and casting rules) to determine the shape, dtype etc. of the result.

**ii) Write to disk** We can in fact write the result direct to disk. 

**EXERCISE**: ``compute`` accepts the same kwargs as the NDArray constructors from the previous tutorial. Write the result to a disk as ``result.b2nd`` and examine the time and memory consumption compared to the case in **i)** above. In principle, no memory is used at all since operands and result are all on disk (why is some memory still used? why is the time taken longer than in **i)**?).

In [ ]:
## YOUR CODE HERE

## Check that the result has been written to disk
!ls -lh result.b2nd

**iiI) Computing slices** We can also compute just a slice of the result.

**EXERCISE**: Look at the documentation of ``compute`` and try to calculate the slice of the result ``slice(2, 200, 25)`` and examine memory consumption and execution time as above. Why is the calculation much faster?
Consider what would happen with a plain NumPy calculation - how could you get a slice of the result faster than calculating the whole result and then slicing?

In [ ]:
## YOUR CODE HERE

What is happening when we call the ``compute`` method? The operands are all NDArray arrays, chunked and stored on disk. When the compute method is called, the expression is executed, chunk-by-chunk, and the result stored, chunk-by-chunk. Hence at any given time, only a small amount of data (a chunk for each operand and the result) must be operated on in memory. The result may be written to memory, chunk-by-chunk, but it is just as easy  (see **ii) Writing to Disk**) to write straight to disk - hence you can operate with very large arrays in a very small memory footprint (a handful of chunks). Moreover, the computation is only performed on the necessary chunks required to give the result slice (see **iii) Computing slices**).

**--------------------Exercise 1------------------**

Blosc2 automatically selects the chunksize and chunk shape when creating arrays to be ajusted to your machine's cache. However, we can modify the chunksizes of ``a`` and ``b`` to be smaller or larger; can you predict how memory usage might change? What about computation time? The latter is probably more difficult to predict.

Extra Credit: Note that we can also specify different compression parameters for the result via the ``cparams`` kwarg of ``compute``.  For example, we can change the codec to `ZLIB`, use the bitshuffle filter, and set the compression level to 9: ``cparams = blosc2.CParams(codec=blosc2.Codec.ZLIB, filters=[blosc2.Filter.BITSHUFFLE], clevel=9)``. What happens to computation time and compression ratio (compare to **i) Computing an expression**) ?


In [ ]:
print(f"Chunks are of size: a - {round(a.chunksize/1024**2,2)} MB,  b - {round(b.chunksize/1024**2,2)} MB")
# Now check what the chunkshapes for a and b are and regenerate them with different chunks
# Try (100, 5000) for a, (250, 1000) for b

#
## YOUR CODE HERE ##
#

# We have to regenerate expression with new operands
print(f"Chunks are now of size: a - {round(a.chunksize/1024**2,2)} MB,  b - {round(b.chunksize/1024**2,2)} MB")
c = a**2 + b**2 + 2 * a * b + 1

# Now redo the experiment ii) Writing to disk and see how memory usage and computation time change

#
## YOUR CODE HERE ##
#

In [ ]:
# Now reset things back to how they were before
a = blosc2.linspace(0, 1, np.prod(shape1), dtype=np.float32, shape=shape1, urlpath="a.b2nd", mode="w")
b = blosc2.linspace(1, 2, np.prod(shape2), dtype=np.float64, shape=shape2, urlpath="b.b2nd", mode="w")
c = a**2 + b**2 + 2 * a * b + 1


#### 2. Returning a NumPy array
Now, let's compute the expression and store the result in a NumPy array.  For this, we will use the `__getitem__` method of the ``LAzyExpr`` ``c``; recall that in the previous tutorial `__getitem__` (i.e. ``arr[1:,4]``) also returned a NumPy array (from decompressing a Blosc2 NDArray). The syntax is the same.

**EXERCISE**: As we did above, examine the memory consumption, execution time and total space occupied by the result when one returns a NumPy array from a lazy expression - both for a) the full result and b)the slice ``slice(2, 200, 25)``. Check the types and shapes of the resulting arrays. What do you notice about memory, time and storage? Does it all make sense?

In [1]:
# a) compute the full expression c and profile it
#YOUR CODE HERE

# b) compute a slice of the expression c and profile it
#YOUR CODE HERE

As you can see, in either case the result is a NumPy array now.

#### Summary
Depending on your needs, you can choose to get the result as a NDArray array or as a NumPy array.  The former is more storage efficient, but the latter is more flexible when interacting with other libraries that do not support NDArray arrays, or for reading out data.

**Note: Functions**
Lazy expressions also support many standard functions (essentially those available in NumPy), such as `sin`, `cos`, `exp`, `log`, etc. as well as reductions like ``sum``, ``mean`` etc.

**EXERCISE**: 
a) Try to write your own lazy expression using the arrays ``a, b`` and some functions in Blosc2 (e.g. ``blosc2.sin``). Examine the type of the result (it should be a ``LazyExpr``). Try to obtain a result slice, first as a Blosc2 NDArray and then as a NumPy array, and check the shapes of the results.

b) Apply ``blosc2.sin`` to the lazy expression from part a). What is the type of the result? Now apply a reduction to the lazy expression from part a). What is the type of the result? 

In [4]:
# a) Write a lazy expression using functions
# YOUR CODE HERE

# b)
# Apply blosc2.sin to lazy expression from a)
#YOUR CODE HERE
# Apply a reduction to lazy expression from a)
#YOUR CODE HERE

As we can see, the result of the reduction expression is not a ``LazyExpr``, but rather a NumPy array (even though no ``__getitem__`` call has been made). This is because reductions in expressions are always executed "eagerly" (i.e. on creation of the lazy expression).

## Part 2) A more practical example

Ok, let's put the computation engine through its paces with some proper computations. We shall set up a 3D grid, represented by 3 1D arrays, and exploit broadcasting to reduce memory footprint.

In [ ]:
# --- Experiment Setup ---
n_frames = 1000  # Raise this for more frames
width, height = np.array((n_frames, n_frames))  # Size of the grid
dtype = np.float64  # Data type for the grid

# --- Coordinate creation ---
x = blosc2.linspace(0, n_frames, n_frames, dtype=dtype)
y = blosc2.linspace(-4 * np.pi, 4 * np.pi, width, dtype=dtype)
z = blosc2.linspace(-4 * np.pi, 4 * np.pi, height, dtype=dtype)
X = blosc2.expand_dims(blosc2.expand_dims(x, 1), 2)  # Shape: (N, 1, 1)
Y = blosc2.expand_dims(blosc2.expand_dims(y, 0), 2)  # Shape: (1, N, 1)
Z = blosc2.expand_dims(blosc2.expand_dims(z, 0), 0)  # Shape: (1, 1, N)

Now lets define a computation result (in 3D) via a function - later on we shall plot it. We can in fact apply NumPy functions to the blosc2 NDArray arrays and the result will still be a lazyexpr.

**EXERCISE**: 

a) Execute ``expr = genexpr(X, Y, Z)`` and profile the memory consumption and execution time of this execution as above (both should be very low). Check the type of ``expr`` and access its metadata (``shape`` and ``dtype``). 

b) How much memory (uncompressed) should the result occupy (i.e. how many GB would ``expr[:]`` load into memory?)? You should be able to work this out just from the ``shape`` and ``dtype`` (HINT: What is the ``dtype`` size?) metadata. The accessing of metadata in a) was instantaneous - if this required calculating the final result, do you think it would be as fast, given the size of the result?)

In [ ]:
# --- Generate computed data ---
def genexpr(a, b, c):
    time_factor = a * b * 0.001
    R = np.sqrt(b**2 + c**2)
    theta = np.arctan2(c, b)
    return np.sin(R * 3 - time_factor * 2) * np.cos(theta * 3)

#a)
# Profile memory usage of expr = genexpr(X, Y, Z)
# YOUR CODE HERE
# Check type, shape and dtype of expr
# YOUR CODE HERE

#b) Calculate size of expr result
# YOUR CODE HERE

Now let's generate an animation which computes the expression slice-by-slice on-the-fly, using the ``LazyExpr`` ``__getitem__`` method:

In [ ]:
def create_video(arr):
    label = 'NumPy' if isinstance(arr, np.ndarray) else 'Blosc2'
    fig = plt.figure(figsize=(8, 8))
    a = arr[:,:,0]
    im = plt.imshow(a, cmap="viridis")
    fig.colorbar(im)
    plt.title("Animated Plot")
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    start_time = time.time()
    def update(frame_num):
        # Evaluate the expression for the current frame on the fly
        a = arr[:, :, frame_num]  # <-------------- perform computation
        im.set_array(a)
        elapsed_time = time.time() - start_time
        plt.title(f"{label}: Frame {frame_num + 1}/{n_frames}, elapsed time = {round(elapsed_time, 2)} s")
        return im,
    ani = FuncAnimation(fig, update, frames=n_frames, interval=10, blit=False, repeat=False)
    plt.show()
    return ani

create_video(expr)


Ok! So our Blosc2 compute engine works well enough to be useful practically - most of the elapsed time is due to matplotlib rendering.

**EXERCISE**: 

Try to rerun the previous cell but replace the computations with just rendering an empty array - the time should be basically the same. Why might it be advantageous to compute slice-by-slice rather than precompute all frames and then access them?

Once you've done all this don't forget to change the``create_video`` function back to the original though!

#### Profiling lazy expressions
We now know that the lazy expression machinery is fast enough to be useful. Now we are going to compare memory usage and computation time to NumPy. In order to do that, we should define a helper function which profiles the ``__getitem__`` method of the lazy expression (which returns a NumPy array) and the NumPy computation.

In [ ]:
# --- Run computation and profile time and memory usage ---
def monitor(BLOSC):
    m0 = getmem()
    t0 = time.time()
    expr = genexpr(X, Y, Z)[:] if BLOSC else genexpr(X[:], Y[:], Z[:])
    dt = time.time()-t0
    if dt > 1e-3:
        print(f'Operation took {round(dt, 3)} s')
    else:
        print(f'Operation took {round(dt * 1000_000, 1)} μs')
    print(f'Result occupies {round((getmem()-m0))} MB')
    return expr

Now let's actually do the computation for the lazy expression, for all frames, and see how the memory usage compares to that of the plain NumPy computation.

**EXERCISE** 

Use the ``monitor`` function to profile the Blosc2 lazy expression computation and check the type and nbytes of the result. Delete the result and now use the monitor function to profile the plain NumPy calculation and again check the type and nbytes of the result (both should be the same).

In [ ]:
print('Profile Blosc2....')
# YOUR CODE HERE

print('--------\nProfile NumPy....')
# YOUR CODE HERE

Blosc2, owing to chunking and caching, makes more effective use of machine architecture, so it actually runs faster in practice! 

**----------------Exercise 2----------------**

a) Run the animation with the result of the NumPy calculation above. Since we precompute all frames, you might expect the animation to run faster. Is this true?
You can go back and regenerate the ``expr`` variable to be a ``LazyExpr`` and rerun the Blosc2 animation cell to double check the comparison of the speeds.

b) Now try to run the animation but pass a computed Blosc2 array as the ``arr`` parameter (rather than either the uncomputed ``LazyExpr`` object or a NumPy array). What happens now?

c) What would happen to memory usage and computation time if, in the ``monitor`` function, we modified the computation to return a Blosc2 NDArray rather than a NumPy array?

## Part 3) Lazy Expressions and Persistent Reductions

Recall that ``blosc2.sum(a)`` returns a computed result, not a lazy expression, which could be undesirable in some cases. Using strings and the ``lazyexpr`` constructor, we can avoid this.

In [ ]:
# Expression that sums arrays
expression = "sum(a + b, axis=1)"
# Define the operands for the expression
operands = {"a": a, "b": b}
# Create a lazy expression
lazy_expression = blosc2.lazyexpr(expression, operands)

**---------------EXERCISE 3----------------**

a) Check the type of ``lazy_expression`` is indeed a ``LazyExpr``. Try computing the full result and profile the computation. Compare it with the time taken for the eagerly executed version which doesn't use a string.

b) Now try computing a slice of ``lazy_expression`` and see if it runs faster than computing the full result. Would the non-string version also be faster? Would the result even be the same?

In [5]:
#a)
# YOUR CODE HERE

#b)
# YOUR CODE HERE


## Part 4) Lazy Expression Operands and Storage

We can save lazy expression operands on-disk and then update them - computing any lazy expression in which they appear then gives a new result, since the computation is peformed every time:

In [ ]:
a = blosc2.arange(0, 10, urlpath="a.b2nd", mode="w")
lexpr = a + 1
print(f"Lazyexpr with old a: {lexpr[:]}")
a = blosc2.arange(10, 20, urlpath="a.b2nd", mode="w")
print(f"Lazyexpr with new a: {lexpr[:]}")  # This will compute with the new a

**-------Exercise 4----------**

Now try the same but *without* saving the operands to disk. What happens now? You should see that the behaviour is different. Compare the result of ``id()`` on ``a`` and the values of the ``lexpr.operands`` dict - what do you see? Try saving and reopening ``lexpr`` to disk when the operands are saved on disk (and when they're not) - what happens then?

In [6]:
a = blosc2.arange(0, 10) # operand stored in memory
print(f"Memory address of a: {id(a)}")

#
## YOUR CODE  HERE
#

NameError: name 'blosc2' is not defined

## Summary

In this section, we have seen how to perform computations with NDArray arrays: how to create lazy expressions, compute them, and save them to disk. Also, we have looked at performing reductions, broadcasting and heavy computations. Lazy expressions allow you to build and compute complex computations from operands that can be in-memory, on-disk or remote (see [`C2Array`](reference/c2array.html)) in a simple and effective way.

In [ ]:
# cleanup
blosc2.remove_urlpath("a.b2nd")
blosc2.remove_urlpath("b.b2nd")
blosc2.remove_urlpath("result.b2nd")